# Master Discogs Database

In [1]:
## Basic stuff
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))

## Python Version
import sys
print("Python: {0}".format(sys.version))


################################################################################
## General Stuff
################################################################################
from ioUtils import saveJoblib, loadJoblib, saveFile, getFile
import urllib
from urllib.parse import quote
from collections import Counter
from searchUtils import findExt, findSubExt, findPatternExt, findNearest
from timeUtils import clock, elapsed
from fsUtils import moveFile, setFile, setDir, setSubDir, isFile, isDir, mkDir
from fileUtils import getFileBasics, getBasename
from listUtils import getFlatList
from time import sleep


################################################################################
## Music Stuff
################################################################################

### MultiArtist
from multiArtist import multiartist

### My Music DB
from myMusicDBMap import myMusicDBMap
from musicDBMap import musicDBMap
from matchDBArtist import matchDBArtist
from masterDBMatchClass import masterDBMatchClass

### Master DB code
from masterdb import masterdb
from mainDB import mainDB


import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Notebook Last Run Initiated: 2020-11-17 19:03:38.116357


# Main DB

In [2]:
### Master DB code
%load_ext autoreload
%autoreload
from masterdb import masterdb
from mainDB import mainDB
maindb = mainDB(mdb=None, create=False, debug=False)
maindb.loadDBDataMap()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=========================   discogs   =========================
=========================   allmusic   =========================
=========================   musicbrainz   =========================
=========================   lastfm   =========================
=========================   rockcorner   =========================
=========================   acebootlegs   =========================
=========================   cdandlp   =========================
=========================   rateyourmusic   =========================
=========================   musicstack   =========================
=========================   discogs   =========================
Summary Statistics For DB: Discogs
    Using Known Artists: False
    Found 753273 ID -> Name entries
    Found 687680 Name -> ID entries
    Found 0 Albums
=========================   allmusic   =========================
Summary Statistics For DB: All

In [3]:
mdbmap = musicDBMap("Music", init=False)

  Loaded 4732 previously matched entries


In [4]:
dbDict = {}
dbDict["AllMusic"]      = "AM"
dbDict["Discogs"]       = "DC"
dbDict["MusicBrainz"]   = "MB"
dbDict["LastFM"]        = "LM"
dbDict["RockCorner"]    = "RC"
dbDict["MusicStack"]    = "MS"
dbDict["CDandLP"]       = "CL"
dbDict["AceBootlegs"]   = "AB"
dbDict["RateYourMusic"] = "RM"
dbDict

{'AllMusic': 'AM',
 'Discogs': 'DC',
 'MusicBrainz': 'MB',
 'LastFM': 'LM',
 'RockCorner': 'RC',
 'MusicStack': 'MS',
 'CDandLP': 'CL',
 'AceBootlegs': 'AB',
 'RateYourMusic': 'RM'}

In [5]:
loadAll = False
if loadAll:
    mdbmaps = {}
    for db in maindb.getDBs():
        print("{0: <30}".format(db), end="")
        mdbmaps[db] = musicDBMap(db, init=False)
    #mdbmaps = {db: musicDBMap(db, init=False) for db in maindb.getDBs()}
else:
    mdbmaps = {}
    #mdbmaps["AllMusic"] = musicDBMap("AllMusic", init=False)
    #mdbmaps["MusicBrainz"] = musicDBMap("MusicBrainz", init=False)
    mdbmaps["Discogs"] = musicDBMap("Discogs", init=False)
    if False:
        mdbmaps["AllMusic"] = musicDBMap("AllMusic", init=False)
        mdbmaps["MusicBrainz"] = musicDBMap("MusicBrainz", init=False)
        mdbmaps["LastFM"] = musicDBMap("LastFM", init=False)
        mdbmaps["RockCorner"] = musicDBMap("RockCorner", init=False)
        mdbmaps["AceBootlegs"] = musicDBMap("AceBootlegs", init=False)
        mdbmaps["MusicStack"] = musicDBMap("MusicStack", init=False)
        mdbmaps["CDandLP"] = musicDBMap("CDandLP", init=False)

  Loaded 34416 previously matched entries


****

# Primary DB Matching

In [6]:
mdbmc = masterDBMatchClass(maindb, mdbmaps)

Loading Artist Names


In [ ]:
matchData = mdbmc.getDBMatchData("Discogs", returnData=False)

In [ ]:
toMatch = mdbmc.getDataToMatch("Discogs", maxValues=1000000, maxAlbums=1000000)

In [ ]:
ddf = mdbmaps["Discogs"].getDF().T

In [ ]:
ddf

In [ ]:
mdbmaps["Discogs"] = musicDBMap("Discogs", init=True)

In [ ]:
toMatchKeys = {primKey[1]: (primKey[0],primKey[1]) for primKey in toMatch["Discogs"]}

In [ ]:
amdf = getDBDF(mdbmc, "AllMusic", dbDict["AllMusic"])

In [ ]:
amdf = getDBDF(mdbmc, "MusicBrainz", dbDict["MusicBrainz"])

In [ ]:
primaryInfo = ("AllMusic", "AM")
primaryInfo = ("MusicBrainz", "MB")
secondaryInfo = ("Discogs", "DC")


In [ ]:
amrcdf = amdf[~amdf[secondaryDBName].isna()][[primaryDBArtistName, primaryDBKey, secondaryDBName]]

In [ ]:
amrcdf

In [ ]:
matchData
mdbmc

# Match Mutual DBs

In [7]:
from difflib import SequenceMatcher
from pandas import Series, DataFrame, isna, isnull

def getDBDF(mdbmc, dbName, dbShort, keepNA=False):
    amdf = DataFrame(mdbmaps[dbName].getDF().T)
    print("Total Size: {0}".format(amdf.shape[0]))
    dbcols = amdf.columns

    matchData  =  mdbmc.getDBMatchData(dbName)
    mAlbDF = DataFrame(Series({primaryKey: len(albums) for primaryKey,albums in matchData.items()}))

    amdf = amdf.join(mAlbDF)
    cols = list(amdf.columns)
    cols[-1] = "{0}Albums".format(dbShort)
    amdf.columns = cols

    amdf.reset_index(inplace=True)
    columns = list(amdf.columns)
    columns[0] = "{0}ArtistName".format(dbShort)
    columns[1] = "{0}Key".format(dbShort)
    amdf.columns = columns
    if keepNA is False:
        amdf = amdf[~amdf[dbName].isna()]

    colOrder = columns[:2] + [columns[-1]] + list(dbcols)
    amdf = amdf[colOrder]

    amdf = amdf.sort_values("{0}Albums".format(dbShort), ascending=False)

    print(" Good Size: {0}".format(amdf.shape[0]))
    return amdf


def checkDBIDMatch(amdf, dbName, dbShort):
    wrong = amdf[amdf["{0}Key".format(dbShort)] != amdf[dbName]]
    idxs  = list(wrong.index)
    print("Found {0} wrongly assigned index".format(len(idxs)))
    return idxs
 
    
#####################################################################################################################
# Fill Matched DB Data From Overlapping Data In A Previously Matched DB
#####################################################################################################################
def mutualMatch(amdf, mdbmaps, primaryInfo, secondaryInfo, debug=False, test=True, ratioCut=0.9):
    primaryDBName  = primaryInfo[0]
    primaryDBShort = primaryInfo[1]
    primaryDBArtistName = "{0}ArtistName".format(primaryDBShort)
    primaryDBKey        = "{0}Key".format(primaryDBShort)

    secondaryDBName  = secondaryInfo[0]
    secondaryDBShort = secondaryInfo[1]

    debug = debug
    amrcdf = amdf[~amdf[secondaryDBName].isna()][[primaryDBArtistName, primaryDBKey, secondaryDBName]]
    for i,row in amrcdf.iterrows():

        ## Primary DB Key --> Secondary ID
        amDBName = row[primaryDBArtistName]
        amDBKey  = row[primaryDBKey]
        rcDBKey  = row[secondaryDBName]

        ## Secondary ID --> Secondary DB Key
        primKey  = mdbmaps[secondaryDBName].getPrimaryKeyFromID(rcDBKey)
        if primKey is None:
            continue
        rcDBName = primKey[0]
        rcDBKey  = primKey[1]


        s = SequenceMatcher(None, str(amDBName), str(rcDBName))

        if debug is True:
            print("{0: <30}{1: <30}{2: <30}{3}".format("", primaryDBName, secondaryDBName, "Match"))
            print("{0: <30}{1: <30}{2: <30}{3}".format("{0} ID   --> {1} ID".format(primaryDBShort, secondaryDBShort), amDBKey, rcDBKey, round(s.ratio(),2)))
            print("{0: <30}{1: <30}{2: <30}".format("{0} Name --> {1} Name".format(primaryDBShort, secondaryDBShort), amDBName, rcDBName))


        amDBData = mdbmaps[primaryDBName].getArtistData(amDBName, amDBKey)
        rcDBData = mdbmaps[secondaryDBName].getArtistData(rcDBName, rcDBKey)

        amRCDBKey = rcDBKey
        rcAMDBKey = rcDBData.getDBID(primaryDBName)
        if debug is True:
            print("{0: <30}{1: <30}{2: <30}".format("{0} ID   --> {1} ID".format(secondaryDBShort, primaryDBShort), str(rcAMDBKey), str(amRCDBKey)))    

        if s.ratio() > ratioCut:
            if rcAMDBKey != amDBKey:
                print("{0: <50}{1: <75}\t--->\t[{2}/{3}]".format("Setting {0} DBData For {1}".format(secondaryDBName,primaryDBName),"[{0: <30} {1: <4} {2: >30}]".format(amDBName,round(s.ratio(),2),rcDBName),amDBKey,rcDBKey))
                if test is True:
                    continue
                mdbmaps[secondaryDBName].addArtistData(rcDBName, rcDBKey, primaryDBName, amDBKey)

        if debug:
            print("\n")

    if test is True:
        return
    mdbmaps[secondaryDBName].save()
    
    
    
#####################################################################################################################
# Fill Fresh DB Data From A Previously Matched DB
#####################################################################################################################
def mutualMatchFromPreviousDBMatch(amdf, mdbmaps, primaryInfo, secondaryInfo, debug=False, test=True, ratioCut=0.9):

    primaryDBName  = primaryInfo[0]
    primaryDBShort = primaryInfo[1]
    primaryDBArtistName = "{0}ArtistName".format(primaryDBShort)
    primaryDBKey        = "{0}Key".format(primaryDBShort)

    secondaryDBName  = secondaryInfo[0]
    secondaryDBShort = secondaryInfo[1]



    amrcdf = amdf[~amdf[secondaryDBName].isna()][[primaryDBArtistName, primaryDBKey, secondaryDBName]]
    for i,row in amrcdf.iterrows():

        ## Primary DB Key --> Secondary ID
        amDBName = row[primaryDBArtistName]
        amDBKey  = row[primaryDBKey]
        rcDBKey  = row[secondaryDBName]

        if amDBKey is None:
            continue

        #if debug is True:
        #    print(amDBName,'\t',amDBKey,'\t',rcDBKey,'\t-->\t',end="")

        ## Secondary ID --> Secondary DB Key
        primKey = toMatchKeys.get(rcDBKey)
        #if debug is True:
        #    print(primKey)
        #primKey  = mdbmaps[secondaryDBName].getPrimaryKeyFromID(rcDBKey)

        if primKey is None:
            continue
        rcDBName = primKey[0]
        rcDBKey  = primKey[1]



        s = SequenceMatcher(None, str(amDBName), str(rcDBName))

        if debug is True:
            print("{0: <30}{1: <30}{2: <30}{3}".format("", primaryDBName, secondaryDBName, "Match"))
            print("{0: <30}{1: <30}{2: <30}{3}".format("{0} ID   --> {1} ID".format(primaryDBShort, secondaryDBShort), amDBKey, rcDBKey, round(s.ratio(),2)))
            print("{0: <30}{1: <30}{2: <30}".format("{0} Name --> {1} Name".format(primaryDBShort, secondaryDBShort), amDBName, rcDBName))

        continue
        if not mdbmaps[secondaryDBName].isKnown(rcDBName, rcDBKey):
            mdbmaps[secondaryDBName].addArtist(rcDBName, rcDBKey)
            mdbmaps[secondaryDBName].addArtistData(rcDBName, rcDBKey, secondaryDBName, rcDBKey)


        amDBData = mdbmaps[primaryDBName].getArtistData(amDBName, amDBKey)
        rcDBData = mdbmaps[secondaryDBName].getArtistData(rcDBName, rcDBKey)

        amRCDBKey = rcDBKey
        rcAMDBKey = rcDBData.getDBID(primaryDBName)
        if debug is True:
            print("{0: <30}{1: <30}{2: <30}".format("{0} ID   --> {1} ID".format(secondaryDBShort, primaryDBShort), str(rcAMDBKey), str(amRCDBKey)))    

        if s.ratio() > ratioCut:
            if rcAMDBKey != amDBKey:
                print("{0: <50}{1: <75}\t--->\t[{2}/{3}]".format("Setting {0} DBData For {1}".format(secondaryDBName,primaryDBName),"[{0: <30} {1: <4} {2: >30}]".format(amDBName,round(s.ratio(),2),rcDBName),amDBKey,rcDBKey))
                if test is True:
                    continue
                mdbmaps[secondaryDBName].addArtistData(rcDBName, rcDBKey, primaryDBName, amDBKey)


        if debug:
            print("\n")

    if test is False:
        mdbmaps[secondaryDBName].save()    
    
    

def isSame(x):
    name  = x.MyArtistName
    match = x["{0}ArtistName".format(dbShort)]
    if not all([name,match]):
        return None
    if any([isna(x) for x in [name,match]]):
        return None

    same = False
    if all([name,match]):
        if str(name) == str(match):
            same = True
    else:
        raise ValueError([name,match])
    return same

def ratio(x):
    name  = x.MyArtistName
    match = x["{0}ArtistName".format(dbShort)]
    same  = x.Same
    if same is None:
        return None
    else:
        if all([name,match]):
            s = SequenceMatcher(None, str(name), str(match))
            return s.ratio()
        return -1.0

In [ ]:
mdbmaps["Discogs"].getDF().T

In [ ]:
amdf = getDBDF(mdbmc, "AllMusic", dbDict["AllMusic"])

In [ ]:
mutualMatch(amdf, mdbmaps, ("AllMusic", "AM"), ("Discogs", "DC"), debug=False, test=True)

# Matching Code

In [8]:
from tqdm import tqdm
from multiprocessing import Pool
from functools import partial
import time

def matchDBArtistWithAlbums(item, *args, **kwargs):    
    #time.sleep(0.0025)

    artistName   = item[0]
    artistID     = item[1]
    artistAlbums = item[2]
    
    mdbMatcher = matchDBArtist(maindb)
    mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
    mcs    = mdbMatcher.findPotentialArtistAlbumMatches()
    retval = [artistName,artistID,mcs]
    return retval
    #result = findNearest(name, artists, 1, kwargs['cutoff'])


def multiProc(func, argument_list, num_processes):
    pool = Pool(processes=num_processes)
    result_list_tqdm = []
    for result in tqdm(pool.imap(func=func, iterable=argument_list), total=len(argument_list)):
        result_list_tqdm.append(result)
    return result_list_tqdm



def saveMapData(mdbmap, result_list):
    for item in result_list:
        artistName = item[0]
        artistID   = item[1]
        mcs        = item[2]
        mdbmap.addArtist(artistName, artistID)
        for db,mc in mcs.items():
            matchID    = mc.matchID
            matchScore = mc.matchScore
            if matchID is not None:
                mdbmap.addArtistData(artistName, artistID, db, matchID)

    mdbmap.save()

In [ ]:
mdbmc = masterDBMatchClass(maindb, mdbmaps)

In [ ]:
toMatch   = mdbmc.getDataToMatch("Discogs", maxValues=500, maxAlbums=1000)

In [ ]:
toMatch

In [ ]:
mdbmc = masterDBMatchClass(maindb, mdbmaps)
for i in range(100):
    #for db in ["AllMusic", "MusicBrainz", "LastFM", "RockCorner", "CDandLP", "RateYourMusic", "MusicStack"]:
    for db in ["Discogs"]: #, "MusicBrainz", "LastFM", "RockCorner", "CDandLP", "RateYourMusic", "MusicStack"]:
        toMatch   = mdbmc.getDataToMatch(db, maxValues=2500, maxAlbums=250)
        
        if len(toMatch) == 0:
            continue
        

        num_processes = 3
        func = matchDBArtistWithAlbums
        pfunc = partial(matchDBArtistWithAlbums, cutoff=0.95) # Giving some arguments for kwargs
        #argument_list = list(inputs.items()) # [random.randint(0, 100) for _ in range(num_jobs)]
        dbName = list(toMatch.keys())[0]
        argument_list = toMatch[dbName]
        if len(argument_list) == 0:
            break
        print("Running imap multiprocessing for {0} artists ...".format(len(argument_list)))
        result_list = multiProc(func=pfunc, argument_list=argument_list,
                                               num_processes=num_processes)


        break
        saveMapData(mdbmaps[dbName], result_list)
        sleep(10)

        #mdbmc.matchMutualMaps()

Loading Artist Names
Loading Artist Albums
ToMatch   -> 2500
MaxAlbums -> 2060
Known     -> 34416
Total     -> 753273
Running imap multiprocessing for 2500 artists ...


 43%|████▎     | 1078/2500 [31:07<1:13:07,  3.09s/it]

In [ ]:
result_list

In [ ]:
toMatch

In [ ]:
mbdf = mdbmaps["MusicBrainz"].getDF().T

In [ ]:
maindb.dbdata["MusicBrainz"]["Utils"].getArtistID("1660fbb6-c55e-4ad3-8342-c424162c30a6")

In [ ]:
maindb.dbdata["MusicBrainz"]["Utils"].getArtistID("c685765c-b778-4ed6-9035-8f7088431765")

In [ ]:
maindb.dbdata["MusicBrainz"]["Utils"].getArtistID("5bf05dd4-3ac1-4b8b-b9a1-163790d10192")

In [ ]:
mdbmaps["MusicBrainz"].addArtist("The Highwomen", "286831209235815715269312378569239355331")
mdbmaps["MusicBrainz"].addArtistData("The Highwomen", "286831209235815715269312378569239355331", "MusicBrainz", "286831209235815715269312378569239355331")
mdbmaps["MusicBrainz"].addArtistData("The Highwomen", "286831209235815715269312378569239355331", "AllMusic", "0003859078")
mdbmaps["MusicBrainz"].save()

In [ ]:
for idx in mbdf.index:
    if idx[0].find("Highway") != -1:
        print(idx)

In [ ]:
toget   = getFile(ifile="togetMB.p")
for primaryKey in toget:
    mdbmaps["MusicBrainz"].removeArtist(primaryKey[0], primaryKey[1])
mdbmaps["MusicBrainz"].save()

In [ ]:
if False:
    results = {}
    for item in matchData[:100]:
        print(item[0])
        mdbMatcher.setArtistInfo(artistName, artistID, artistAlbums)
        #mdbMatcher.findPotentialArtistNameMatches()
        #mc = mdbMatcher.findPotentialArtistAlbumMatchesByDB('Discogs')
        mcs = mdbMatcher.findPotentialArtistAlbumMatches()
        results[(artistName,artistID)] = {db: [mc.matchID, mc.matchScore] for db, mc in mcs.items()}

In [ ]:
df = mdbmap.getDF().T

In [ ]:
from pandas import Series
matchData  = mdbmc.getDBMatchData("AllMusic")
sortedData = Series({primaryKey: len(albums) for primaryKey,albums in matchData.items()}).sort_values(ascending=False).to_dict()

In [ ]:
sortedData

In [ ]:
matchData